In [ ]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("..")

from ggqpy import construct_Chebyshev_quadratures
from ggqpy.utils import Interval, FunctionFamilySymbolic
from ggqpy.discretize import compress_sequence_of_functions, Discretizer, construct_A_matrix
from ggqpy.optimize import QuadOptimizer
from ggqpy.visualize import plot_points, visualise_diagonal_dropoff

In [ ]:
eps_disc = 1e-10
eps_comp = 1e2*eps_disc
eps_quad = 1e-6
I = Interval(1e-8,1)

In [ ]:
F = FunctionFamilySymbolic.polynomials_and_singularity(I, order = 5, number_of_polynomials = 20)
ex_f_expr, ex_f = F.draw_function()
display(ex_f_expr)
display(F.functions_symbolic[1])

In [ ]:
print(F.integral(ex_f_expr))

In [ ]:
print("Example function from function space")
display(ex_f_expr)
xx = np.linspace(I.a,I.b,1000)
functions_to_plot = 5
plt.subplot(1, 2, 1)
plt.title("Example functions")
plt.plot(xx,ex_f(xx))
plt.subplot(1, 2, 2);
xx2 = np.linspace((I.a + I.b)/4,I.b,1000)
plt.plot(xx2,ex_f(xx2));

Discretization and compression

In [ ]:
disc = Discretizer(eps_disc,min_length=1e-7, interpolation_degree=30)
x_disc, w_disc = disc.adaptive_discretization(F)
U_disc, rank = compress_sequence_of_functions(F.functions_lambdas, x_disc, w_disc, eps_comp)
u_family = disc.interpolate_piecewise_legendre(U_disc)

In [ ]:
[print(i) for i in disc.intervals]
# plt.plot(endpoints)

In [ ]:
print("Functions before compression:", len(F.functions_lambdas))
print("Functions after compression:", u_family.number_of_functions)
A = construct_A_matrix(x_disc,w_disc,F.functions_lambdas)
visualise_diagonal_dropoff(A, eps_comp)

# Chebyshev Rule

In [ ]:
(x_cheb,), w_cheb = construct_Chebyshev_quadratures((x_disc,),w_disc,U_disc)


In [ ]:
print(w_disc.sum())
print(w_cheb.sum())

In [ ]:
print("Number of points in original discretization:", len(x_disc))
print("Number of points in Chebyshev rule:", len(x_cheb))
print("Absolute integral error for discretization:", abs(F.integral(ex_f_expr)-ex_f(x_disc)@w_disc))
print("Absolute integral error for chebyshev:", abs(F.integral(ex_f_expr)-ex_f(x_cheb)@w_cheb))

In [ ]:
print(F.functions_symbolic)

Nonlinear optimization

In [ ]:
r = U_disc.T@w_disc

In [ ]:
plt.plot(x_cheb)

In [ ]:


opt = QuadOptimizer(u_family,r)
x,w = opt.reduce_quadrature(x_cheb, w_cheb, eps_quad)
print(x_cheb)

In [ ]:
print("Quadrature length: ", len(x))
print("Absolute error:", abs(F.integral(ex_f_expr)-ex_f(x)@w))

Comparison with Gauss-Legendre

In [ ]:
fig, ax = plt.subplots()
x_gl,w_gl = np.polynomial.legendre.leggauss(len(x))
w_gl = w_gl*0.5*I.length()
x_gl = I.translate(x_gl)
ax.axhline(y=0, c="orange")
ax.stem(x_gl, w_gl, markerfmt = 'bo', basefmt="orange", label=r"Gauss-Legendre nodes")
ax.stem(x, w, markerfmt = 'go', basefmt="orange", label=r"New quadrature nodes")
ax.legend()
print("Gauss-Legendre",np.around(x_gl,3))
print("New quadrature",np.around(sorted(x),3))

In [ ]:
symx = sym.Symbol("x",real=True)
expr = symx**2
display(expr)
f = sym.lambdify(symx, expr, "numpy")
print("Gauss-Legendre error", abs(F.integral(expr) - f(x_gl)@w_gl))
print("New quadrature error", abs(F.integral(expr) - f(x)@w))


In [ ]:
expr = 1/symx + symx**2
display(expr)
f = sym.lambdify(symx, expr, "numpy")
print("Gauss-Legendre error", abs(F.integral(expr) - f(x_gl)@w_gl))
print("New quadrature error", abs(F.integral(expr) - f(x)@w))